# 공모전

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Selenium 옵션 설정 (백그라운드 실행)
chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 경로 설정
driver_path ="C:\\Users\\최라윤\\OneDrive\\바탕 화면\\infostar\\chromedriver-win64\\chromedriver.exe"

service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 베이스 URL
BASE_URL = "https://www.campuspick.com/contest"
driver.get(BASE_URL)
time.sleep(3)  # 페이지 로딩 대기

# 🔹 1️⃣ 카테고리 목록 자동 가져오기
category_elements = driver.find_elements(By.CSS_SELECTOR, "a[href^='/contest?category=']")
category_list = []

for category in category_elements:
    category_url = category.get_attribute("href")
    category_id = category.get_attribute("data-category")
    category_name = category.text.strip()
    
    if category_id and category_url:  # 카테고리 ID와 URL이 있으면 추가
        category_list.append({"id": category_id, "name": category_name, "url": category_url})

print(f"🔍 {len(category_list)}개의 카테고리 발견됨.")

# 결과 저장 리스트
contest_data = []

# 🔹 2️⃣ 각 카테고리별 공모전 크롤링
for category in category_list:
    category_id = category["id"]
    category_name = category["name"]
    category_url = category["url"]
    
    print(f"📂 카테고리 크롤링 시작: {category_name} ({category_id})")
    driver.get(category_url)
    time.sleep(3)

    # 페이지 스크롤 다운 (모든 공모전 로딩)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 🔹 3️⃣ 공모전 목록 가져오기
    contests = driver.find_elements(By.CSS_SELECTOR, "div.list > div.item a.top")
    contest_links = [contest.get_attribute("href") for contest in contests]

    print(f"✅ {len(contest_links)}개의 공모전 발견됨.")

    # 최대 100개의 공모전만 크롤링
    for contest_url in contest_links[:100]:
        driver.get(contest_url)
        time.sleep(2)

        try:
            # 제목
            title = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()

            # 주최/주관
            try:
                host = driver.find_element(By.CSS_SELECTOR, "p.company").text.strip()
            except:
                host = "정보 없음"

            # 접수기간
            try:
                period = driver.find_element(By.XPATH, "//h2[contains(text(),'접수 기간')]/following-sibling::p[@class='indent']").text.strip()
            except:
                period = "정보 없음"

            # D-day
            try:
                d_day = driver.find_element(By.CSS_SELECTOR, "p.dday.highlight").text.strip()
            except:
                d_day = "정보 없음"

            # 조회수, 저장 수, 댓글 수
            try:
                view_count = driver.find_element(By.CSS_SELECTOR, "span.viewcount").text.strip()
            except:
                view_count = "정보 없음"

            try:
                save_count = driver.find_element(By.CSS_SELECTOR, "a[href*='menu=save'] span.count").text.strip()
            except:
                save_count = "0"

            try:
                comment_count = driver.find_element(By.CSS_SELECTOR, "a[href*='menu=comment'] span.count").text.strip()
            except:
                comment_count = "0"

            # 포스터 이미지 URL
            try:
                image_element = driver.find_element(By.CSS_SELECTOR, "figure")
                image_url = image_element.value_of_css_property("background-image").replace('url("', '').replace('")', '')
            except:
                image_url = "정보 없음"

            # 상금 정보
            try:
                prize = driver.find_element(By.CSS_SELECTOR, "#container > div:nth-child(3) > p:nth-child(3)").text.strip()
            except:
                prize = "정보 없음"

            # 상세 내용
            try:
                description_element = driver.find_element(By.CSS_SELECTOR, "article.description")
                description = description_element.text.strip().replace("\n", " ")
            except:
                description = "정보 없음"

            # 데이터 저장
            contest_data.append({
                "카테고리": category_name,
                "제목": title,
                "주최/주관": host,
                "접수기간": period,
                "D-day": d_day,
                "조회수": view_count,
                "저장": save_count,
                "댓글": comment_count,
                "포스터": image_url,
                "상세 링크": contest_url,
                "상금": prize,
                "상세 내용": description
            })

            print(f"✅ 크롤링 완료: {title}")

        except Exception as e:
            print(f"❌ 오류 발생: {e}")

    print(f"📂 카테고리 '{category_name}' 크롤링 완료.\n")

# WebDriver 종료
driver.quit()

# 결과 출력 (예시로 첫 번째 공모전 데이터 출력)
if contest_data:
    print("첫 번째 공모전 데이터:")
    for key, value in contest_data[0].items():
        print(f"{key}: {value}")
else:
    print("수집된 공모전 데이터가 없습니다.")


🔍 9개의 카테고리 발견됨.
📂 카테고리 크롤링 시작: 기획/아이디어 (101)
✅ 2020개의 공모전 발견됨.
✅ 크롤링 완료: 제1회 CAIND 청년 국제개발 에세이 공모
✅ 크롤링 완료: 제69회 신문의 날 신문홍보 캐릭터를 공모합니다.
✅ 크롤링 완료: 제95회 춘향제 홍보 콘텐츠 & 아이디어 공모전
✅ 크롤링 완료: 실패, 성공 그 이상의 시도 챠챠챠 5기 모집
✅ 크롤링 완료: 2025 탄소중립 실천 캘리그림 공모전
✅ 크롤링 완료: 2025 서울조각페스티벌 제2회 서울조각상 공모
✅ 크롤링 완료: DN프릭스 굿즈 개발 아이디어 공모전
✅ 크롤링 완료: 제6회 건설•교통신기술 슬로건 및 사진 공모전
✅ 크롤링 완료: 2025 대한민국 대학생 패키징 공모전징 공모전
✅ 크롤링 완료: 청년바다마을 조성 설계 아이디어 공모전
✅ 크롤링 완료: 청년 바다마을 조성 설계 아이디어 공모전
✅ 크롤링 완료: [재단법인 미래와소프트웨어] 정보보안 SW 웹/앱 개발 공모전
✅ 크롤링 완료: 2025년 제23회 포천시민의날 시민공모전
✅ 크롤링 완료: 2025년 광진구 빅데이터 분석 공모전
✅ 크롤링 완료: [예술X기술] 융복합콘텐츠 기획·제작 과정 <합톤> 참가자 모집!
✅ 크롤링 완료: 2025 중소기업 안전보건 혁신 사업 아이디어 공모전
✅ 크롤링 완료: 2025 드림라이언
✅ 크롤링 완료: 2025 한복디자인 프로젝트 공모전
✅ 크롤링 완료: 2025 전기에너지 국민 아이디어 공모
✅ 크롤링 완료: 2025 학생 철도 창의 작품전
✅ 크롤링 완료: 제1회 글로벌 숏폼 드라마 시나리오 공모전
✅ 크롤링 완료: [로레알코리아] 로레알2025 브랜드스톰 공모전
✅ 크롤링 완료: 15초로 나만의 기업가 정신을 세상에 알려봐! 🚀
✅ 크롤링 완료: 제1회 PMI한국챕터 대학생 프로젝트 경연대회 [조기마감]
✅ 크롤링 완료: 2025년 강남 환경교육 프로그램 공모전
✅ 크롤링 완료: 제3회 시장경제 스피치대회 안내
✅ 크롤링 완료: 일동제약그룹 신사업 플랫폼 기획/마케팅

In [ ]:
df = pd.DataFrame(contest_data)
df.to_excel('C:\\Users\\최라윤\\OneDrive\\바탕 화면\\infostar\\contest_data.xlsx', index=False, engine='openpyxl')


# 대외활동

In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Selenium 옵션 설정 (백그라운드 실행)
chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 경로 설정
driver_path ="C:\\Users\\최라윤\\OneDrive\\바탕 화면\\infostar\\chromedriver-win64\\chromedriver.exe"

service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 베이스 URL
BASE_URL = "https://www.campuspick.com/activity"
driver.get(BASE_URL)
time.sleep(3)  # 페이지 로딩 대기

# 🔹 1️⃣ 카테고리 목록 자동 가져오기
category_elements = driver.find_elements(By.CSS_SELECTOR, "a[href^='/activity?category=']")
category_list = []

for category in category_elements:
    category_url = category.get_attribute("href")
    category_id = category.get_attribute("data-category")
    category_name = category.text.strip()
    
    if category_id and category_url:  # 카테고리 ID와 URL이 있으면 추가
        category_list.append({"id": category_id, "name": category_name, "url": category_url})

print(f"🔍 {len(category_list)}개의 카테고리 발견됨.")

# 결과 저장 리스트
activity_data = []

# 🔹 2️⃣ 각 카테고리별 대외활동 크롤링
for category in category_list:
    category_id = category["id"]
    category_name = category["name"]
    category_url = category["url"]
    
    print(f"📂 카테고리 크롤링 시작: {category_name} ({category_id})")
    driver.get(category_url)
    time.sleep(3)

    # 페이지 스크롤 다운 (모든 대외활동 로딩)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 🔹 3️⃣ 대외활동 목록 가져오기
    activities = driver.find_elements(By.CSS_SELECTOR, "div.list > div.item a.top")
    activity_links = [activity.get_attribute("href") for activity in activities]

    print(f"✅ {len(activity_links)}개의 대외활동 발견됨.")

    # 최대 100개의 대외활동만 크롤링
    for activity_url in activity_links[:100]:
        driver.get(activity_url)
        time.sleep(2)

        try:
            # 제목
            title = driver.find_element(By.CSS_SELECTOR, "body > section.title > div > div > h1").text.strip()

            # 주최
            try:
                host = driver.find_element(By.CSS_SELECTOR, "body > section.title > div > div > p.company").text.strip()
            except:
                host = "정보 없음"

            # 팀원 모집 수
            try:
                team_count = driver.find_element(By.CSS_SELECTOR, "body > section.tab > div > a:nth-child(2) > span.teamcount.count").text.strip()
            except:
                team_count = "0"

            # 댓글 수
            try:
                comment_count = driver.find_element(By.CSS_SELECTOR, "body > section.tab > div > a:nth-child(3) > span.commentcount.count").text.strip()
            except:
                comment_count = "0"

            # 디데이
            try:
                d_day = driver.find_element(By.CSS_SELECTOR, "#container > div:nth-child(1) > p.dday.highlight").text.strip()
            except:
                d_day = "정보 없음"

            # 접수 기간
            try:
                period = driver.find_element(By.CSS_SELECTOR, "#container > div:nth-child(1) > p.indent").text.strip()
            except:
                period = "정보 없음"

            # 조회수 (추가적으로 찾을 필요 있음)
            try:
                view_count = driver.find_element(By.CSS_SELECTOR, "span.viewcount").text.strip()
            except:
                view_count = "정보 없음"

            # 포스터 이미지 URL
            try:
                poster_element = driver.find_element(By.CSS_SELECTOR, "#container > div.poster > img")
                poster_url = poster_element.get_attribute("src")
            except:
                poster_url = "정보 없음"

            # 상세 내용
            try:
                description_element = driver.find_element(By.CSS_SELECTOR, "#container > div:nth-child(4) > article")
                description = description_element.text.strip().replace("\n", " ")
            except:
                description = "정보 없음"

            # 데이터 저장
            activity_data.append({
                "카테고리": category_name,
                "제목": title,
                "주최": host,
                "조회수": view_count,
                "D-day": d_day,
                "접수기간": period,
                "팀원 모집": team_count,
                "댓글 수": comment_count,
                "포스터": poster_url,
                "상세 링크": activity_url,
                "상세 내용": description
            })

            print(f"✅ 크롤링 완료: {title}")

        except Exception as e:
            print(f"❌ 오류 발생: {e}")

    print(f"📂 카테고리 '{category_name}' 크롤링 완료.\n")

# WebDriver 종료
driver.quit()

# 🔹 4️⃣ DataFrame으로 변환 및 CSV 파일로 저장
df = pd.DataFrame(activity_data)
csv_file_path = "activity_data.csv"
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

print(f"📁 데이터가 '{csv_file_path}' 파일로 저장되었습니다.")


🔍 7개의 카테고리 발견됨.
📂 카테고리 크롤링 시작: 국내봉사 (201)
✅ 3620개의 대외활동 발견됨.
✅ 크롤링 완료: 클라랑스 대학생 CSR 서포터즈 클라미 8기 모집
✅ 크롤링 완료: 종로장애인복지관 윤슬대학생봉사단 2기 모집
✅ 크롤링 완료: 꿈의숲종합사회복지관 홍보서포터즈 모집
✅ 크롤링 완료: [광진구청소년상담복지센터]2025년 대학생동아리 with U(위드유) 모
✅ 크롤링 완료: 🔥 재능 공유 프로젝트 꼼지락 4기 모집 🔥
✅ 크롤링 완료: [두암종합사회복지관] 광주광역시 북구 아동 멘토 모집
✅ 크롤링 완료: 2025년 시각장애인가정 자녀학습지도 학습도우미 추가모집
✅ 크롤링 완료: 성프란치스꼬장애인종합복지관 ‘열린마을 발굴단’ 모집
✅ 크롤링 완료: 옥스팜 크루 1기
✅ 크롤링 완료: 익명의 고민에 손편지 답장을 전해주실 '온기우체부'를 모집해요💌
✅ 크롤링 완료: ✏️ 손글씨 기부 봉사활동을 함께할 따뜻한 분들을 모집해요.
✅ 크롤링 완료: 제3기 공정병역 지킴이 모집 공고
✅ 크롤링 완료: 서울노인복지센터) 2025 영상서포터즈단 모집
✅ 크롤링 완료: 🌈2025년 9기 대학생기획단 '유스토리' 추가모집🌈
✅ 크롤링 완료: 사상구청소년유해환경감시단원 모집
✅ 크롤링 완료: 사상구청소년수련관 청소년참여위원회 '사상타울' 모집
✅ 크롤링 완료: 사상구청소년수련관 문화기획 서포터즈 세레니티 모집
✅ 크롤링 완료: 🚨분당판교청소년수련관 문화기획단 T-Ma!n단원 모집‼️
✅ 크롤링 완료: 2025년 대학생 자원봉사 서포터즈 '띵동' 봉사자 모집
✅ 크롤링 완료: 2025년 기아 프렌토링 신규 멘토 모집
✅ 크롤링 완료: 대학생 환경서포터즈 “에코나래 2기” 모집
✅ 크롤링 완료: 청춘어람 14기 모집
✅ 크롤링 완료: [대한민국 대학생 교육기부단] 대교단 전라제주권역본부 25기 모집
✅ 크롤링 완료: [청년문간] 2025 세대공감잇다 - 청년이 써가는 어르신 자서전
✅ 크롤링 완료: KT&G복지재단 아름드리 수피아 9기 모집
✅ 크롤링 완료

In [58]:
df.to_excel('C:\\Users\\최라윤\\OneDrive\\바탕 화면\\infostar\\activity_data.xlsx', index=False, engine='openpyxl')
